In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bearing-data/B021_2.mat
/kaggle/input/bearing-data/B028_3.mat
/kaggle/input/bearing-data/IR007_3.mat
/kaggle/input/bearing-data/IR014_3.mat
/kaggle/input/bearing-data/OR00712_2.mat
/kaggle/input/bearing-data/IR028_2.mat
/kaggle/input/bearing-data/B014_3.mat
/kaggle/input/bearing-data/IR014_0.mat
/kaggle/input/bearing-data/IR007_2.mat
/kaggle/input/bearing-data/OR0073_1.mat
/kaggle/input/bearing-data/Normal_3.mat
/kaggle/input/bearing-data/IR028_0.mat
/kaggle/input/bearing-data/B021_1.mat
/kaggle/input/bearing-data/B028_1.mat
/kaggle/input/bearing-data/IR014_1.mat
/kaggle/input/bearing-data/Normal_1.mat
/kaggle/input/bearing-data/OR0076_1.mat
/kaggle/input/bearing-data/B007_1.mat
/kaggle/input/bearing-data/B014_0.mat
/kaggle/input/bearing-data/OR0146_2.mat
/kaggle/input/bearing-data/B014_1.mat
/kaggle/input/bearing-data/OR0216_3.mat
/kaggle/input/bearing-data/B028_0.mat
/kaggle/input/bearing-data/OR02112_2.mat
/kaggle/input/bearing-data/B007_3.mat
/kaggle/input/bearing-dat

In [28]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.signal import spectrogram
from scipy.stats import kurtosis, skew
from tqdm import tqdm
import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns

In [29]:
sampling_rate = 12000
window_sec = 5
out_shape = (128, 128)
data_path = "/kaggle/input/bearing-data"  


def classify_file(filename):
    if "IR" in filename:
        return 1  
    elif "B0" in filename:
        return 3  
    elif "OR" in filename:
        return 2  
    elif "Normal" in filename or "" in filename:
        return 0  
    else:
        return -1  

In [30]:
def segment_signal(signal, fs=12000, window_sec=5):
    win_len = fs * window_sec
    segments = []
    for start in range(0, len(signal), win_len):
        chunk = signal[start:start+win_len]
        if len(chunk) == win_len:
            segments.append(chunk)
    return segments

def segment_to_spectrogram(segment, fs=12000, out_shape=(128, 128)):
    f, t, Sxx = spectrogram(segment, fs, nperseg=256, noverlap=128)
    Sxx_db = 10 * np.log10(Sxx + 1e-8)
    out = np.zeros(out_shape)
    h, w = min(Sxx_db.shape[0], out_shape[0]), min(Sxx_db.shape[1], out_shape[1])
    out[:h, :w] = Sxx_db[:h, :w]
    return out

In [31]:
X, y = [], []
mat_files = glob.glob(os.path.join(data_path, "*.mat"))

for filepath in tqdm(mat_files):
    try:
        fname = os.path.basename(filepath)
        label = classify_file(fname)
        if label == -1:
            continue  # skip unknown types

        data = loadmat(filepath)
        key = [k for k in data.keys() if "DE_time" in k][0]  # auto-detect key
        signal = data[key].ravel()

        segments = segment_signal(signal, fs=sampling_rate, window_sec=window_sec)
        for seg in segments:
            spec = segment_to_spectrogram(seg, fs=sampling_rate, out_shape=out_shape)
            X.append(spec)
            y.append(label)

    except Exception as e:
        print(f"Error processing {filepath}: {e}")


X = np.array(X)
y = np.array(y)
X = X[..., np.newaxis]  


print("Final dataset:")
print("Total samples:", X.shape[0])
print("Shape of each image:", X.shape[1:])
print("Label distribution:", pd.Series(y).value_counts().sort_index().to_dict())

100%|██████████| 64/64 [00:00<00:00, 145.20it/s]

Final dataset:
Total samples: 148
Shape of each image: (128, 128, 1)
Label distribution: {0: 28, 1: 32, 2: 56, 3: 32}


In [32]:

X, y = [], []
mat_files = glob.glob(os.path.join(data_path, "*.mat"))

for filepath in tqdm(mat_files):
    try:
        fname = os.path.basename(filepath)
        label = classify_file(fname)
        if label == -1:
            continue  # skip unknown types

        data = loadmat(filepath)
        key = [k for k in data.keys() if "DE_time" in k][0]  # auto-detect key
        signal = data[key].ravel()

        segments = segment_signal(signal, fs=sampling_rate, window_sec=window_sec)
        for seg in segments:
            # Original
            spec = segment_to_spectrogram(seg, fs=sampling_rate, out_shape=out_shape)
            X.append(spec)
            y.append(label)

            # Augmentation 1: Add Gaussian noise
            noisy = seg + np.random.normal(0, 0.01, size=seg.shape)
            noisy_spec = segment_to_spectrogram(noisy, fs=sampling_rate, out_shape=out_shape)
            X.append(noisy_spec)
            y.append(label)

            # Augmentation 2: Time shift (roll)
            shifted = np.roll(seg, 200)
            shifted_spec = segment_to_spectrogram(shifted, fs=sampling_rate, out_shape=out_shape)
            X.append(shifted_spec)
            y.append(label)

    except Exception as e:
        print(f"Error processing {filepath}: {e}")


X = np.array(X)
y = np.array(y)
X = X[..., np.newaxis]  

# Summary
print("Final dataset:")
print("Total samples:", X.shape[0])
print("Shape of each image:", X.shape[1:])
print("Label distribution:", pd.Series(y).value_counts().sort_index().to_dict())


100%|██████████| 64/64 [00:01<00:00, 46.68it/s]

Final dataset:
Total samples: 444
Shape of each image: (128, 128, 1)
Label distribution: {0: 84, 1: 96, 2: 168, 3: 96}


In [33]:
# --- Cross-Validation Block with Improvements ---
print("\n--- 5-Fold Cross Validation ---")
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y), 1):
    print(f"Fold {fold}")
    unique, counts = np.unique(y[test_idx], return_counts=True)
    print("Test set label distribution:", dict(zip(unique, counts)))

    model = tf.keras.Sequential([
        tf.keras.Input(shape=(128, 128, 1)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X[train_idx], y[train_idx], epochs=10, verbose=0)
    y_pred = model.predict(X[test_idx]).argmax(axis=1)
    acc = accuracy_score(y[test_idx], y_pred)
    print(f"Accuracy: {acc:.4f}\n")
    accuracies.append(acc)

print("Cross-validation accuracies:", [round(a, 4) for a in accuracies])
print("Average accuracy:", round(np.mean(accuracies), 4))


--- 5-Fold Cross Validation ---
Fold 1
Test set label distribution: {0: 17, 1: 19, 2: 33, 3: 20}
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
Accuracy: 1.0000

Fold 2
Test set label distribution: {0: 17, 1: 20, 2: 33, 3: 19}
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
Accuracy: 0.3708

Fold 3
Test set label distribution: {0: 17, 1: 19, 2: 34, 3: 19}
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
Accuracy: 0.9888

Fold 4
Test set label distribution: {0: 17, 1: 19, 2: 34, 3: 19}
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
Accuracy: 1.0000

Fold 5
Test set label distribution: {0: 16, 1: 19, 2: 34, 3: 19}
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
Accuracy: 0.9886

Cross-validation accuracies: [1.0, 0.3708, 0.9888, 1.0, 0.9886]
Average accuracy: 0.8696
